# PerfectFit: AI-Based Outfit Recommendation
This notebook implements a Genetic Algorithm to generate optimal outfit recommendations based on user preferences such as dress code, color palette, comfort level, and budget.

In [1]:
# Import necessary libraries
import random
import pandas as pd
import matplotlib.pyplot as plt

In [2]:

class Item:
    """
    Represents an item of clothing with attributes like category, name, price, dress code, color, and comfort level.
    """
    def __init__(self, Category, ItemName, Price, DressCode, Color, ComfortLevel):
        self.Category = Category
        self.ItemName = ItemName
        self.Price = Price
        self.DressCode = DressCode
        self.Color = Color
        self.ComfortLevel = ComfortLevel

    def __repr__(self):
        return f"Item({self.Category}, {self.ItemName}, {self.Price}, {self.DressCode}, {self.Color}, {self.ComfortLevel})"


In [3]:
class Outfit:
    """
    Represents an outfit composed of a top, bottom, shoes, neck accessory, and purse.
    """
    def __init__(self, top, bottom, shoes, neck, purse):
        self.top = top
        self.bottom = bottom
        self.shoes = shoes
        self.neck = neck
        self.purse = purse
        self.fitness = None

    def __repr__(self):
        return (f"Outfit(Top: {self.top.ItemName}, "
                f"Bottom: {self.bottom.ItemName}, "
                f"Shoes: {self.shoes.ItemName}, "
                f"Neck: {self.neck.ItemName}, "
                f"Purse: {self.purse.ItemName}, "
                f"Fitness: {self.fitness})")


In [4]:
def Create_Initial_Population(population_size, category_items):
    """
    Creates an initial population of outfits by randomly selecting items from each category.
    """
    population = []
    for _ in range(population_size):
        selected_top = random.choice(category_items.get('Top'))
        selected_bottom = random.choice(category_items.get('Bottom'))
        selected_shoes = random.choice(category_items.get('Shoes'))
        selected_neck = random.choice(category_items.get('Neck'))
        selected_purse = random.choice(category_items.get('Purse'))

        outfit = Outfit(selected_top, selected_bottom, selected_shoes, selected_neck, selected_purse)
        population.append(outfit)

    return population


In [5]:
def fitness_function(outfit, dressCode, palette, comfort, budget):
    """
    Calculates the fitness of an outfit based on dress code, color palette, comfort level, and budget.
    """
    sumDress = sumPallete = sumComfort = sumPrice = 0

    for item in [outfit.top, outfit.bottom, outfit.shoes, outfit.neck, outfit.purse]:
        sumDress += 1 if item.DressCode.casefold() == dressCode.casefold() else 0
        sumPallete += 1 if item.Color.casefold() == palette.casefold() else 0
        sumComfort += 1 if item.ComfortLevel == comfort else 0
        sumPrice += item.Price

    withinBudget = 1 if sumPrice <= budget else 0
    weighted_sum = 0.35 * sumDress + 0.15 * sumPallete + 0.15 * sumComfort + 0.35 * withinBudget
    weighted_sum = weighted_sum / 3.6  # Normalize to a maximum of 1.0

    outfit.fitness = weighted_sum


In [6]:
def select_parents(population):
    """
    Selects two parents from the population using binary tournament selection.
    """
    parents = []
    for _ in range(2):
        a = random.choice(population)
        b = random.choice(population)
        parents.append(a if a.fitness > b.fitness else b)
    return parents


In [7]:
def two_point_crossover(parent1, parent2, crossover_point1, crossover_point2):
    """
    Performs two-point crossover between two parents to produce two children.
    """
    crossover_points = sorted([crossover_point1, crossover_point2])

    child1_items = [
        getattr(parent1, attr) if i <= crossover_points[0] or i > crossover_points[1] else getattr(parent2, attr)
        for i, attr in enumerate(['top', 'bottom', 'shoes', 'neck', 'purse'], start=1)
    ]

    child2_items = [
        getattr(parent2, attr) if i <= crossover_points[0] or i > crossover_points[1] else getattr(parent1, attr)
        for i, attr in enumerate(['top', 'bottom', 'shoes', 'neck', 'purse'], start=1)
    ]

    child1 = Outfit(*child1_items)
    child2 = Outfit(*child2_items)

    return child1, child2


In [8]:
def mutate(outfit, mutation_rate, category_items):
    """
    Mutates an outfit by randomly replacing one of its items with another from the same category.
    """
    if random.random() < mutation_rate:
        item_to_mutate = random.choice(['top', 'bottom', 'shoes', 'neck', 'purse'])
        current_item = getattr(outfit, item_to_mutate)
        possible_items = [item for item in category_items.get(item_to_mutate.capitalize()) if item != current_item]
        setattr(outfit, item_to_mutate, random.choice(possible_items))


In [9]:
def find_optimal_outfit(population_size, dressCode, palette, comfort, budget):
    """
    Finds the optimal outfit using a genetic algorithm.
    """
    random.seed(42)
    population = Create_Initial_Population(population_size, category_items)

    for outfit in population:
        fitness_function(outfit, dressCode, palette, comfort, budget)

    max_generations = 20000
    best_outfit = max(population, key=lambda o: o.fitness)

    return best_outfit


In [11]:
df = pd.read_csv('search_space.csv')

category_items = {}
for index, row in df.iterrows():
    item = Item(row['Category'], row['ItemNamse'], row['Price'], row['Dress Code'], row['Color'], row['Comfort Level'])
    if row['Category'] not in category_items:
        category_items[row['Category']] = []
    category_items[row['Category']].append(item)

entered_dressCode = input('Enter dress code (Casual, Sportswear, Business, Evening): ')
entered_color = input('Enter color palette (Dark, Bright): ')
entered_comfort = int(input('Enter comfort level (1-5): '))
entered_budget = int(input('Enter budget (SAR): '))

best_outfit = find_optimal_outfit(10, entered_dressCode, entered_color, entered_comfort, entered_budget)

print('\nSuggested outfit:')
print('Top:', best_outfit.top.ItemName)
print('Bottom:', best_outfit.bottom.ItemName)
print('Shoes:', best_outfit.shoes.ItemName)
print('Neck:', best_outfit.neck.ItemName)
print('Purse:', best_outfit.purse.ItemName)
print(f'Fitness: {best_outfit.fitness:.2f}')


Enter dress code (Casual, Sportswear, Business, Evening): casual
Enter color palette (Dark, Bright): Bright
Enter comfort level (1-5): 4
Enter budget (SAR): 4000

Suggested outfit:
Top: Polo Shirt
Bottom: Skirt
Shoes: Sandals
Neck: Sports Scarf
Purse: Canvas Bag
Fitness: 0.58
